<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>



# Demo 8.3: Web Scraping

INSTRUCTIONS:

- Run the cells
- Observe and understand the results

# Web Scraping in Python (using BeautifulSoup)

# Basics HTML
Before starting with the code, let’s understand the basics of HTML and some rules of scraping.

## HTML tags
Below is the source code for a simple HTML webpage.

    <!DOCTYPE html>  
    <html>  
        <head>
        </head>
        <body>
            <h1> First Scraping </h1>
            <p> Hello World </p>
        <body>
    </html>
    
This is the basic syntax of an HTML webpage. Every `<tag>` serves a block inside the webpage:
1. `<!DOCTYPE html>` HTML documents must start with a type declaration.
2. The HTML document is contained between `<html>` and `</html>`.
3. The meta and script declaration of the HTML document is between `<head>` and `</head>`.
4. The visible part of the HTML document is between `<body>` and `</body>` tags.
5. Title headings are defined with the `<h1>` through `<h6>` tags.
6. Paragraphs are defined with the `<p>` tag.

Other useful tags include `<a>` for hyperlinks, `<table>` for tables, `<tr>` for table rows, and `<td>` for table columns.

Also, HTML tags sometimes come with `id` or `class` attributes. The `id` attribute specifies a unique id for an HTML tag and the value must be unique within the HTML document. The `class` attribute is used to define equal styles for HTML tags with the same class. We can make use of these ids and classes to help us locate the data we want.

## Scraping Rules
1. **Always** check a website’s **Terms and Conditions** before you scrape it. Be careful to read the statements about legal use of data. Usually, the retrieved data should not be used for commercial purposes.
2. **Do not** request data from the website too aggressively with a program (also known as spamming), as this may break the website. Make sure the program behaves in a reasonable manner (i.e. acts like a human). One request for one webpage per second is good practice.
3. The layout of a website may change from time to time, so make sure to revisit the site and rewrite the code as needed

## Inspecting the Page
Let’s take one page from the **Memory Alpha** website as an example.

To investigate some relationships let's get the links from this page.

Open the web page on [Prinadora](https://memory-alpha.fandom.com/wiki/Prinadora) with the browser and inspect it.

Hover the cursor on the text and follow the shaded box surrounding the main text.

From the result, check the navigation menu text inside a couple of levels of HTML tags, which is `<div class="fandom-sticky-header">` → `<nav class="fandom-community-header__local-navigation">`.

This section of the data is what we focus on.

In [1]:
## Import Libraries
import regex as re

from urllib.parse import unquote
import urllib3
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

### Define the content to retrieve (webpage's URL)

In [2]:
# specify the url
quote_page = 'http://memory-alpha.wikia.com/wiki/Prinadora'

### Retrieve the page
- Require Internet connection

In [3]:
# query the website and return the html to the variable ‘page’
http = urllib3.PoolManager()
r = http.request('GET', quote_page)
if r.status == 200:
    page = r.data
    print('Type of the variable \'page\':', page.__class__.__name__)
    print('Page Retrieved. Request Status: %d, Page Size: %d' % (r.status, len(page)))
else:
    print('Some problem occurred. Request Status: %s' % r.status)

Type of the variable 'page': bytes
Page Retrieved. Request Status: 200, Page Size: 402214


### Convert the stream of bytes into a BeautifulSoup representation

In [4]:
# parse the html using beautiful soup and store in variable `soup`
soup = BeautifulSoup(page, 'html.parser')
print('Type of the variable \'soup\':', soup.__class__.__name__)

Type of the variable 'soup': BeautifulSoup


### Check the content
- The HTML source
- Includes all tags and scripts
- Can be long!

In [5]:
print(soup.prettify()[:1000])

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Prinadora | Memory Alpha | Fandom
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"7180ce9b846bc64c","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Prinadora","wgTitle":"Prinadora","wgCurRevisionId":2846125,"wgRevisionId":2846125,"wgArticleId":8581,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Ferengi"],"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgRelevantPageName":"Prinadora","wgRelevantArticleId":8581,"wgIsProbablyEditable":!1,"wgRelevantPageIsProb

### Check the HTML's Title

In [6]:
print('Title tag :%s:' % soup.title)
print('Title text:%s:' % soup.title.string)

Title tag :<title>Prinadora | Memory Alpha | Fandom</title>:
Title text:Prinadora | Memory Alpha | Fandom:


###  nav tag
- This page uses the tag `nav` for navigation links

        <nav class="fandom-community-header__local-navigation">

In [7]:
tag = 'nav'
nav = soup.find_all(tag)[0]
print('Type of the variable \'article\':', nav.__class__.__name__)

Type of the variable 'article': Tag


### Find the content under the 'nav' tag

In [8]:
# show the first 500 characters after removing redundant newlines
print(re.sub(r'\n\n+', '\n', nav.text)[:500])


 Explore
 
 Main Page
 Discuss
All Pages
Community
Interactive Maps
Portals
 
People
 
Bajorans
Borg
Ferengi
Humans
Klingons
Romulans
Vulcans
Starfleet personnel
Society & Culture
 
Borg
Cardassian Union
Dominion
Ferengi Alliance
Federation
Klingon Empire
Romulan Empire
Vulcan
Science
 
Alpha Quadrant
Beta Quadrant
Gamma Quadrant
Delta Quadrant
Technology
 
Spacecraft
Starships
Spacecraft classes
Starship classes
Stations
All technology
The Alternate Reality
 
History 
People
Places
Things
Spac


### Find the links in the text

In [9]:
# identify the type of tag to retrieve
tag = 'a'
# create a list with the links from the `<a>` tag
tag_list = [t.get('href') for t in nav.find_all(tag)]
tag_list

['#',
 'https://memory-alpha.fandom.com/wiki/Portal:Main',
 '/f',
 'https://memory-alpha.fandom.com/wiki/Special:AllPages',
 'https://memory-alpha.fandom.com/wiki/Special:Community',
 'https://memory-alpha.fandom.com/wiki/Special:AllMaps',
 'https://memory-alpha.fandom.com/wiki/Category:Memory_Alpha_portals',
 'https://memory-alpha.fandom.com/wiki/Portal:People',
 'https://memory-alpha.fandom.com/wiki/Category:Bajorans',
 'https://memory-alpha.fandom.com/wiki/Category:Borg',
 'https://memory-alpha.fandom.com/wiki/Category:Ferengi',
 'https://memory-alpha.fandom.com/wiki/Category:Humans',
 'https://memory-alpha.fandom.com/wiki/Category:Klingons',
 'https://memory-alpha.fandom.com/wiki/Category:Romulans',
 'https://memory-alpha.fandom.com/wiki/Category:Vulcans',
 'https://memory-alpha.fandom.com/wiki/Category:Starfleet_personnel',
 'https://memory-alpha.fandom.com/wiki/Portal:Society_and_Culture',
 'https://memory-alpha.fandom.com/wiki/Borg',
 'https://memory-alpha.fandom.com/wiki/Cardas

In [10]:
# keep only the links to the wiki itself
tag_list = [t[37:] for t in tag_list if (t) and ('/wiki/' in t)]
tag_list

['Portal:Main',
 'Special:AllPages',
 'Special:Community',
 'Special:AllMaps',
 'Category:Memory_Alpha_portals',
 'Portal:People',
 'Category:Bajorans',
 'Category:Borg',
 'Category:Ferengi',
 'Category:Humans',
 'Category:Klingons',
 'Category:Romulans',
 'Category:Vulcans',
 'Category:Starfleet_personnel',
 'Portal:Society_and_Culture',
 'Borg',
 'Cardassian_Union',
 'Dominion',
 'Ferengi_Alliance',
 'United_Federation_of_Planets',
 'Klingon_Empire',
 'Romulan_Star_Empire',
 'Vulcan',
 'Portal:Science',
 'Alpha_Quadrant',
 'Beta_Quadrant',
 'Gamma_Quadrant',
 'Delta_Quadrant',
 'Portal:Technology',
 'Category:Spacecraft',
 'Category:Starships',
 'Category:Spacecraft_classes',
 'Category:Starship_classes',
 'Category:Stations',
 'Category:Technology',
 'Portal:Alternate_Reality',
 'Alternate_reality',
 'Category:Alternate_reality_inhabitants',
 'Category:Locations_(alternate_reality)',
 'Category:Alternate_reality',
 'Category:Spacecraft_(alternate_reality)',
 'Category:Starfleet_pers

### Create a filter for undesired links

In [11]:
filter  = '(%s)' % '|'.join([
    'Category:',
    'File:',
    'Help:',
    'Memory_Alpha:',
    'Portal:',
    'action=',
    'Special:',
    'Star_Trek:',
    'Star_Trek_',
    'Talk:'
])
# remove the links that are found in the filter
tag_list = [t for t in tag_list if not re.search(filter, t)]
tag_list

['Borg',
 'Cardassian_Union',
 'Dominion',
 'Ferengi_Alliance',
 'United_Federation_of_Planets',
 'Klingon_Empire',
 'Romulan_Star_Empire',
 'Vulcan',
 'Alpha_Quadrant',
 'Beta_Quadrant',
 'Gamma_Quadrant',
 'Delta_Quadrant',
 'Alternate_reality',
 'Studio_model',
 'Retroactive_continuity',
 'Deleted_scene',
 'DVD',
 'Blu-ray_Disc',
 'Digital_formats',
 'Novels',
 'Comics',
 'Calendars',
 'Jonathan_Archer',
 'T%27Pol',
 'Charles_Tucker_III',
 'Malcolm_Reed',
 'Phlox',
 'Hoshi_Sato',
 'Travis_Mayweather',
 'Thy%27lek_Shran',
 'Maxwell_Forrest',
 'Enterprise_(NX-01)',
 'Michael_Burnham',
 'Saru',
 'Paul_Stamets',
 'Sylvia_Tilly',
 'Hugh_Culber',
 'Nhan',
 'Adira_Tal',
 'Cleveland_Booker',
 'Philippa_Georgiou_(mirror)',
 'USS_Discovery',
 'Christopher_Pike',
 'Una',
 'James_T._Kirk',
 'Spock',
 'Leonard_McCoy',
 'Montgomery_Scott',
 'Hikaru_Sulu',
 'Nyota_Uhura',
 'Pavel_Chekov',
 'Christine_Chapel',
 'USS_Enterprise_(NCC-1701)',
 'USS_Enterprise_(NCC-1701-A)',
 'James_T._Kirk_(alternate_

### Remove duplicates

In [12]:
tag_list = list(set(tag_list))
tag_list

['Studio_model',
 'Nhan',
 'Retroactive_continuity',
 'Kes',
 'Montgomery_Scott_(alternate_reality)',
 'Deleted_scene',
 'Montgomery_Scott',
 'Chakotay',
 'Raffaela_Musiker',
 'Drednok',
 'USS_Enterprise_(NCC-1701)',
 'Murf',
 'Tom_Paris',
 'Sam_Rutherford',
 'Agnes_Jurati',
 'Blu-ray_Disc',
 'Una',
 'Thy%27lek_Shran',
 'Carol_Freeman',
 'USS_Cerritos',
 'USS_Protostar',
 'Julian_Bashir',
 'Saru',
 'Dominion',
 'USS_Enterprise_(NCC-1701-D)',
 'Paul_Stamets',
 'Philippa_Georgiou_(mirror)',
 'Odo',
 'Beckett_Mariner',
 'Shaxs',
 'Comics',
 'Ezri_Dax',
 'Benjamin_Sisko',
 'Adira_Tal',
 'T%27Ana',
 'Narissa',
 'Hoshi_Sato',
 'Enterprise_(NX-01)',
 'Jonathan_Archer',
 'Deep_Space_9',
 'Leonard_McCoy',
 'Novels',
 'Quark',
 'Leonard_McCoy_(alternate_reality)',
 'Alpha_Quadrant',
 'T%27Pol',
 'D%27Vana_Tendi',
 'Jake_Sisko',
 'Maxwell_Forrest',
 'Data',
 'DVD',
 'James_T._Kirk_(alternate_reality)',
 'Malcolm_Reed',
 'Travis_Mayweather',
 'Romulan_Star_Empire',
 'Harry_Kim',
 'Pavel_Chekov_(al

### Convert escaped sequences

In [13]:
tag_list = [unquote(t) for t in tag_list]
tag_list

['Studio_model',
 'Nhan',
 'Retroactive_continuity',
 'Kes',
 'Montgomery_Scott_(alternate_reality)',
 'Deleted_scene',
 'Montgomery_Scott',
 'Chakotay',
 'Raffaela_Musiker',
 'Drednok',
 'USS_Enterprise_(NCC-1701)',
 'Murf',
 'Tom_Paris',
 'Sam_Rutherford',
 'Agnes_Jurati',
 'Blu-ray_Disc',
 'Una',
 "Thy'lek_Shran",
 'Carol_Freeman',
 'USS_Cerritos',
 'USS_Protostar',
 'Julian_Bashir',
 'Saru',
 'Dominion',
 'USS_Enterprise_(NCC-1701-D)',
 'Paul_Stamets',
 'Philippa_Georgiou_(mirror)',
 'Odo',
 'Beckett_Mariner',
 'Shaxs',
 'Comics',
 'Ezri_Dax',
 'Benjamin_Sisko',
 'Adira_Tal',
 "T'Ana",
 'Narissa',
 'Hoshi_Sato',
 'Enterprise_(NX-01)',
 'Jonathan_Archer',
 'Deep_Space_9',
 'Leonard_McCoy',
 'Novels',
 'Quark',
 'Leonard_McCoy_(alternate_reality)',
 'Alpha_Quadrant',
 "T'Pol",
 "D'Vana_Tendi",
 'Jake_Sisko',
 'Maxwell_Forrest',
 'Data',
 'DVD',
 'James_T._Kirk_(alternate_reality)',
 'Malcolm_Reed',
 'Travis_Mayweather',
 'Romulan_Star_Empire',
 'Harry_Kim',
 'Pavel_Chekov_(alternate_

### Convert underscore to space

In [14]:
tag_list = [re.sub('_', ' ', t) for t in tag_list]
tag_list

['Studio model',
 'Nhan',
 'Retroactive continuity',
 'Kes',
 'Montgomery Scott (alternate reality)',
 'Deleted scene',
 'Montgomery Scott',
 'Chakotay',
 'Raffaela Musiker',
 'Drednok',
 'USS Enterprise (NCC-1701)',
 'Murf',
 'Tom Paris',
 'Sam Rutherford',
 'Agnes Jurati',
 'Blu-ray Disc',
 'Una',
 "Thy'lek Shran",
 'Carol Freeman',
 'USS Cerritos',
 'USS Protostar',
 'Julian Bashir',
 'Saru',
 'Dominion',
 'USS Enterprise (NCC-1701-D)',
 'Paul Stamets',
 'Philippa Georgiou (mirror)',
 'Odo',
 'Beckett Mariner',
 'Shaxs',
 'Comics',
 'Ezri Dax',
 'Benjamin Sisko',
 'Adira Tal',
 "T'Ana",
 'Narissa',
 'Hoshi Sato',
 'Enterprise (NX-01)',
 'Jonathan Archer',
 'Deep Space 9',
 'Leonard McCoy',
 'Novels',
 'Quark',
 'Leonard McCoy (alternate reality)',
 'Alpha Quadrant',
 "T'Pol",
 "D'Vana Tendi",
 'Jake Sisko',
 'Maxwell Forrest',
 'Data',
 'DVD',
 'James T. Kirk (alternate reality)',
 'Malcolm Reed',
 'Travis Mayweather',
 'Romulan Star Empire',
 'Harry Kim',
 'Pavel Chekov (alternate 

###Order the list

In [15]:
tag_list.sort()
tag_list

['Adira Tal',
 'Agnes Jurati',
 'Alpha Quadrant',
 'Alternate reality',
 "B'Elanna Torres",
 'Beckett Mariner',
 'Benjamin Sisko',
 'Beta Quadrant',
 'Beverly Crusher',
 'Blu-ray Disc',
 'Borg',
 'Brad Boimler',
 'Calendars',
 'Cardassian Union',
 'Carol Freeman',
 'Chakotay',
 'Charles Tucker III',
 'Christine Chapel',
 'Christopher Pike',
 'Christopher Pike (alternate reality)',
 'Cleveland Booker',
 'Comics',
 'Cristóbal Rios',
 "D'Vana Tendi",
 'DVD',
 "Dal R'El",
 'Data',
 'Deanna Troi',
 'Deep Space 9',
 'Deleted scene',
 'Delta Quadrant',
 'Digital formats',
 'Dominion',
 'Drednok',
 'Elnor',
 'Enterprise (NX-01)',
 'Ezri Dax',
 'Ferengi Alliance',
 'Gamma Quadrant',
 'Geordi La Forge',
 'Gwyn',
 'Harry Kim',
 'Hikaru Sulu',
 'Hikaru Sulu (alternate reality)',
 'Hoshi Sato',
 'Hugh Culber',
 'Jack Ransom',
 'Jadzia Dax',
 'Jake Sisko',
 'James T. Kirk',
 'James T. Kirk (alternate reality)',
 'Jankom Pog',
 'Jean-Luc Picard',
 'Jonathan Archer',
 'Julian Bashir',
 'Kathryn Janewa



---



---



> > > > > > > > > © 2023 Institute of Data


---



---



